In [2]:
pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 12.6 MB/s eta 0:00:01
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1671632 sha256=b137cbd96aed602f6510114cd4a95bddcab3ed15eb8b693cc72c61a89952c8c7
  Stored in directory: /home/azureuser/.cache/pip/wheels/de/9a/41/6a57bf37eb7b50de7f8c7ca9d7053bebe0ea7c7c9bae9fa293
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [2]:
azure = True
g_colab = False

In [3]:
import pandas as pd
import numpy as np

In [4]:
if azure:
    path_bcd ="./dataset FC/RatingBook/ratings.csv"
    path_ml  = "./dataset FC/ML/ml-100k/rating_ml.csv"
    path_rdc = "./dataset FC/RDC/rating_final.csv"
    
    
if g_colab:
    from google.colab import drive 
    drive.mount('/content/gdrive')
    path_bcd ="/content/gdrive/My Drive/Monografia/dataset FC/RatingBook/ratings.csv"
    path_ml = "/content/gdrive/My Drive/Monografia/dataset FC/ML/ml-100k/rating_ml.csv"
    path_rdc = "/content/gdrive/My Drive/Monografia/dataset FC/RDC/rating_final.csv"

In [5]:
df_book=pd.read_csv(path_bcd)
df_book = df_book.rename(columns = {"book_id":"itemid","user_id":"userid"})

qr = df_book[["userid","rating"]].groupby("userid").count() 

print('Escala Minima Rating:',np.min(df_book["rating"]))
print('Escala maxima Rating:',np.max(df_book["rating"]))
print("Numero de avaliaçoes",len(df_book))
print("Numero de Usuários",len(df_book.userid.unique()))
print("Numero de itens",len(df_book.itemid.unique()))
df_book.head()

Escala Minima Rating: 1
Escala maxima Rating: 5
Numero de avaliaçoes 981756
Numero de Usuários 53424
Numero de itens 10000


,itemid,userid,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [6]:
df_rdc=pd.read_csv(path_rdc)
print("Numero de avaliaçoes",len(df_rdc))
print("Numero de Usuários",len(df_rdc.userID.unique()))
print("Numero de itens",len(df_rdc.placeID.unique()))
df_rdc['userID'] = pd.to_numeric(df_rdc['userID'].map(lambda x: x.replace('U','')))
df_rdc = df_rdc.rename(columns = {"userID":"userid","placeID":"itemid"})
df_rdc = df_rdc.drop(columns = {"food_rating","service_rating"})

#trocando a escala de 0 a 2 para 1 a 3.
df_rdc["rating"] = df_rdc["rating"] + 1

print("Numero de avaliaçoes",len(df_rdc))
print('Escala Minima Rating:',np.min(df_rdc["rating"]))
print('Escala maxima Rating:',np.max(df_rdc["rating"]))
df_rdc.head()

Numero de avaliaçoes 1161
Numero de Usuários 138
Numero de itens 130
Numero de avaliaçoes 1161
Escala Minima Rating: 1
Escala maxima Rating: 3


,userid,itemid,rating
0,1077,135085,3
1,1077,135038,3
2,1077,132825,3
3,1077,135060,2
4,1068,135104,2


In [7]:
#Normalização do Rating entre 1 a 5
max_rating = np.max(df_rdc["rating"]) 
df_rdc["rating"] = (df_rdc["rating"]*4) / max_rating + 1
df_rdc["rating"] = df_rdc["rating"].round(2)
df_rdc.head()

,userid,itemid,rating
0,1077,135085,5.00
1,1077,135038,5.00
2,1077,132825,5.00
3,1077,135060,3.67
4,1068,135104,3.67


In [8]:
df_ml=pd.read_csv(path_ml,sep= ";")
print('Escala Minima Rating:',np.min(df_ml["rating"]))
print('Escala maxima Rating:',np.max(df_ml["rating"]))
print("Numero de avaliaçoes",len(df_ml))
df_ml.head()

Escala Minima Rating: 1
Escala maxima Rating: 5
Numero de avaliaçoes 100000


,userid,itemid,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import get_dataset_dir
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader, KNNBaseline, accuracy
from surprise.model_selection import train_test_split
import pandas as pd
import numpy  as np
import time

In [8]:

import time

start_time = time.clock()

print (time.clock() - start_time, "seconds")

4.399999999993298e-05 seconds


In [10]:
#Gerando as Sementes para gerar as mesmas amostras


# Ler os dados sobre rating 
reader_ml  = Reader(rating_scale=(1, 5))
reader_bcd  = Reader(rating_scale=(1, 5))
reader_rdc  = Reader(rating_scale=(1, 5))

data_ml = Dataset.load_from_df(df_ml[['userid','itemid','rating']], reader_ml)
data_book = Dataset.load_from_df(df_book[['userid','itemid','rating']], reader_bcd)
data_rdc = Dataset.load_from_df(df_rdc[['userid','itemid','rating']], reader_rdc)
 
# Treinamento com 70% da massa de dados e deixar 30% para fazer a validação
trainset_ml,  testset_ml  = train_test_split(data_ml,   test_size=.30,random_state=42)
trainset_bcd, testset_bcd = train_test_split(data_book, test_size=.30,random_state=42)
trainset_rdc, testset_rdc = train_test_split(data_rdc,  test_size=.30,random_state=42)


my_sim_option_pearson = {'name':'pearson', 'user_based':True}


t_ccp_ml_inicial = time.clock()
KNN_ml_pearson  = KNNBasic(sim_options = my_sim_option_pearson, verbose = True)
KNN_ml_pearson.fit(trainset_ml)
t_ccp_ml_final = time.clock() - t_ccp_ml_inicial


bcd_ccp_init = time.clock()
KNN_bcd_pearson = KNNBasic(sim_options = my_sim_option_pearson, verbose = True)
KNN_bcd.fit(trainset_bcd)
bcd_ccp_final = time.clock() - bcd_ccp_init


rdc_ccp_init = time.clock()
KNN_rdc_pearson = KNNBasic(sim_options = my_sim_option_pearson, verbose = True)
KNN_rdc_pearson.fit(trainset_rdc)
rdc_ccp_final = time.clock() - rdc_ccp_init

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:249: RuntimeWarning: invalid value encountered in sqrt
  sim = construction_func[name](*args)


In [11]:
#predição base de Test
KNN_predictions_ml_pearson  = KNN_ml_pearson.test(testset_ml)
KNN_predictions_bcd_pearson  = KNN_bcd.test(testset_bcd)
KNN_predictions_rdc_pearson = KNN_rdc_pearson.test(testset_rdc)

In [12]:
# Verificar o valor do erro quadrático médico para verificar a qualidade do  modelo.
print("rmse MovieLens:",accuracy.rmse(KNN_predictions_ml_pearson))
print("rmse Book:",accuracy.rmse(KNN_predictions_bcd_pearson))
print("rmse Restaurant:",accuracy.rmse(KNN_predictions_rdc_pearson))

RMSE: 1.0213
rmse MovieLens: 1.0213474337685258
RMSE: 1.0316
rmse Book: 1.0315524316589013
RMSE: 1.0987
rmse Restaurant: 1.0986704580052973


In [13]:
# Verificar o valor do erro quadrático médico para verificar a qualidade do  modelo.
print("mae MovieLens:",accuracy.mae(KNN_predictions_ml_pearson))
print("mae Book:",accuracy.mae(KNN_predictions_bcd_pearson))
print("mae Restaurant:",accuracy.mae(KNN_predictions_rdc_pearson))

MAE:  0.8099
mae MovieLens: 0.8098674173933262
MAE:  0.8129
mae Book: 0.8129046586572023
MAE:  0.9030
mae Restaurant: 0.903028380705733


### Implementação para a Similaridade Cosseno 


Nessa etapa foi implementado a Fc baseado em memória, usando como métrica de avaliação a similaridade cosseno. 

In [14]:
my_sim_option_cosine = {'name':'cosine', 'user_based':True}


KNN_ml_cosine  = KNNBasic(sim_options = my_sim_option_cosine, verbose = True)
KNN_bcd_cosine = KNNBasic(sim_options = my_sim_option_cosine, verbose = True)
KNN_rdc_cosine = KNNBasic(sim_options = my_sim_option_cosine, verbose = True)


# Criando o Modelo
KNN_ml_cosine.fit(trainset_ml)
KNN_bcd_cosine.fit(trainset_bcd)
KNN_rdc_cosine.fit(trainset_rdc)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
#predição base de Test
KNN_predictions_ml_cosine  = KNN_ml_cosine.test(testset_ml)
KNN_predictions_bcd_cosine = KNN_bcd_cosine.test(testset_bcd)
KNN_predictions_rdc_cosine = KNN_rdc_cosine.test(testset_rdc)

In [16]:
# Verificar o valor do erro quadrático médico para verificar a qualidade do  modelo.
print("rmse MovieLens:",accuracy.rmse(KNN_predictions_ml_cosine))
print("rmse Book:",accuracy.rmse(KNN_predictions_bcd_cosine))
print("rmse Restaurant:",accuracy.rmse(KNN_predictions_rdc_cosine))

RMSE: 1.0244
rmse MovieLens: 1.0243627525567949
RMSE: 0.9812
rmse Book: 0.9811851621398898
RMSE: 1.1115
rmse Restaurant: 1.1114581838800037


In [17]:

# Verificar o valor do erro quadrático médico para verificar a qualidade do  modelo.
print("mae MovieLens:",accuracy.mae(KNN_predictions_ml_cosine))
print("mae Book:",accuracy.mae(KNN_predictions_bcd_cosine))
print("mae Restaurant:",accuracy.mae(KNN_predictions_rdc_cosine))

MAE:  0.8099
mae MovieLens: 0.8098724379694351
MAE:  0.7785
mae Book: 0.7784777010284563
MAE:  0.9162
mae Restaurant: 0.9161679502019547


### Calculando o Recall para Correlação de Pearson

In [18]:
from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [19]:
# Escolhendo Corte como 3.5

#predição base de Test
k = 10
threshold = 3.5

pearson_precisions_ml, pearson_recalls_ml = precision_recall_at_k(KNN_predictions_ml_pearson,k=k, threshold=threshold)

pearson_precisions_bcd, pearson_recalls_bcd = precision_recall_at_k(KNN_predictions_bcd_pearson,k=k, threshold=threshold)

pearson_precisions_rdc, pearson_recalls_rdc = precision_recall_at_k(KNN_predictions_rdc_pearson,k=k, threshold=threshold)


print('Movielens ',sum(rec for rec in pearson_recalls_ml.values()) / len(pearson_recalls_ml))

print('Books ',sum(rec for rec in pearson_recalls_bcd.values()) / len(pearson_recalls_bcd))

print('RDC', sum(rec for rec in pearson_recalls_rdc.values()) / len(pearson_recalls_rdc))

Movielens  0.48568138028244323
Books  0.769918055441254
RDC 0.8167256985438804


## Calculando o Recall para a Similaridade Cosseno

In [20]:
# Escolhendo Corte como 3.5

#predição base de Test
k = 50
threshold = 3.5

cosine_precisions_ml, cosine_recalls_ml = precision_recall_at_k(KNN_predictions_ml_cosine,k=k, threshold=threshold)

cosine_precisions_bcd, cosine_recalls_bcd = precision_recall_at_k(KNN_predictions_bcd_cosine,k=k, threshold=threshold)

cosine_precisions_rdc, cosine_recalls_rdc = precision_recall_at_k(KNN_predictions_rdc_cosine,k=k, threshold=threshold)


print('Movielens ',sum(rec for rec in cosine_recalls_ml.values()) / len(cosine_recalls_ml))

print('Books ',sum(rec for rec in cosine_recalls_bcd.values()) / len(cosine_recalls_bcd))

print('RDC', sum(rec for rec in cosine_recalls_rdc.values()) / len(cosine_recalls_rdc))

Movielens  0.7279800322214656
Books  0.7774767093214215
RDC 0.7292798110979928


In [21]:
#KNN_predictions_rdc_cosine

In [22]:
 # First map the predictions to each user.
user_est_true = defaultdict(list)
for uid, _, true_r, est, _ in KNN_predictions_rdc_cosine:
    user_est_true[uid].append((est, true_r))